In [54]:
from django.contrib.postgres.aggregates import StringAgg
from django_cte import CTEQuerySet
from django_cte import With 
from django.db.models import CharField
from django.db.models import OuterRef
from django.db.models import Subquery
from django.db.models import Value
from django.db.models import When
from django.db.models.functions.comparison import Cast
from django.db.models.functions import Coalesce
from django.db.models.functions import Concat
from django.db.models.functions.comparison import Cast
from django.db.models.functions.comparison import NullIf
from django.db.models.functions.text import Trim

from common.models.utils import override_current_transaction


workbasket_pk = 349
measure_sid = 20185730

workbasket = WorkBasket.objects.get(pk=workbasket_pk)
current_transaction = workbasket.current_transaction


def get_measure_queryset(current_transaction, sid):
    # Counterintuitively, TrackedModelQuerySet.get_latest_version() fails
    # to get the latest (or any) version of a Model instance that is not in
    # an approved WorkBasket - i.e. its status is in appoved_status.
    # This is related to VersionGroup.current_version not being set.
    # VersionGroup.current_version is only set in TrackedModel.save() when the
    # containing WorkBasket.status is one of WorkflowStatus.approved_statuses()).
    # Therefore, TrackedModelQuerySet.approved_up_to_transaction() is the way
    # it's done in Tamato's Views and Forms.
    m_qs = (
        Measure.objects.filter(sid=sid)
            .approved_up_to_transaction(
                current_transaction
            )
    )
    
    return m_qs
    
    
def get_components_queryset(current_transaction, measure_qs):
    # NOTE: It would be preferable to get a VersionGroup QuerySet
    # of each Measure.version_group in measure_qs, but how?
    version_group_ids = measure_qs.values_list("version_group", flat=True).distinct()
    #print(f"*** version_group_ids = {version_group_ids}")
    components = MeasureComponent.objects.filter(
        component_measure__version_group__id__in=version_group_ids
    )
    return components


def with_duty_sentence(measure_qs):
    measure_components = MeasureComponent.objects.filter(
        # sid or version_group can be used to filter on measure:
        #component_measure__sid=OuterRef("sid"),
        component_measure__version_group=OuterRef("version_group"),
        # Latest component transaction for this measure:
        transaction__id__lte=OuterRef("transaction__id"),
    ).order_by("-transaction__id")

    measure_qs = measure_qs.annotate(
        components_tranx_id=Subquery(
           measure_components[:1].values('transaction_id')
       )
    )

    # TODO:
    # Now create a CTE to join with the measure_qs on components_tranx_id.
    
    return measure_qs


measure_qs = get_measure_queryset(current_transaction, measure_sid)
print(f"*** measure_qs = {measure_qs}")

measure_qs = with_duty_sentence(measure_qs)
for m in measure_qs:
    print(f"*** sid:{m.sid}:")
    print(f"    components_tranx_id={m.components_tranx_id}")
    print(f"    components={m.components.all()}")


*** measure_qs = <MeasuresQuerySet [<Measure: 20185730>]>
*** sid:20185730:
    components_tranx_id=5118158
    components=<TrackedModelQuerySet [<MeasureComponent: component_measure__sid=20185730, duty_expression__sid=1>]>
